## Import data

In [95]:
numDataCols = 10

import csv
import numpy as np

def findStrElement(element, array):
    for item in array:
        if str(item) == str(element):
            print(item)
            return item

def quarterObjectExists(quarter, quarterArray):        
    for lQuarter in quarterArray:
        if str(quarter) == str(lQuarter):
            return True
    return False

In [57]:
quarters = {}
counter = 0;
    
    
    
with open('digitalStandardsTabSeparatedData.txt', newline = '') as data:
    data_reader = csv.reader(data, delimiter='\t')
    
    for data in data_reader:
        counter+=1
        if counter == 1: ## Skip the header
            continue
        
        quarterStr = data[0] + "-Q" + data[1]
        data_arr = [
            int(data[2]), int(data[3]), int(data[4]), 
            int(data[5]), int(data[6]), int(data[7]), 
            int(data[8]), int(data[9]), int(data[10]), 
            int(data[11])
        ]
        
        if quarterStr in quarters:
            quarters[quarterStr] = np.append(quarters[quarterStr], data_arr)
        else:
            quarters[quarterStr] = np.array(data_arr)

## Chart Overall Alignment

In [111]:
import plotly.graph_objects as go
import datetime

def getSumDigitalStandardsArr(npDictionary):
    totalsArr = []
    
    for key in npDictionary:
        total = sum(quarters[key])
        numOfSubmissions = (len(quarters[key])/numDataCols)
        totalsArr.append(total/numOfSubmissions)
    
    return totalsArr
    

In [112]:
quartersArr = []
for quarter in quarters:
    quartersArr.append(str(quarter))
    
x = quartersArr
y = getSumDigitalStandardsArr(quarters)

fig = go.Figure(data=[go.Scatter(x=x, y=y)])

fig.update_layout(xaxis_range=quartersArr)
fig.show()

{'2020-Q1': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1]), '2020-Q2': array([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1])}
[1.5, 2.3333333333333335]
